# Proximity Trees
This is an algorithm made up by Bart Goethals. It doesn't exist yet. It's bsed on another algorithm created for other purposes.

You can read section 3 of [this paper](https://arxiv.org/pdf/1808.10594.pdf) (skip 3.1).
Lucas, B., Shifaz, A., Pelletier, C., O’Neill, L., Zaidi, N., Goethals, B., ... & Webb, G. I. (2019). Proximity forest: an effective and scalable distance-based classifier for time series. Data Mining and Knowledge Discovery, 33(3), 607-635.

## The model

In [125]:
import numpy as np


class Branch:
    def __init__(self, exemplar, class_label, subtree):
        self.exemplar = exemplar  # An object, if from a node it is closest to this exemplar, it goes to given tree
        self.subtree = subtree  # Should be an internal node or a leaf node

        self.class_label = class_label  # The class label of the leaf node, not required for the algorithm, but might be useful for debuggin


class TreeNode:
    def predict(self, data):
        raise NotImplementedError()

    def print(self, depth):
        raise NotImplementedError()


class InternalNode(TreeNode):
    def __init__(self, data_x, data_y, groups):
        self.measure = lambda x, y: np.linalg.norm(x - y)
        self.branches = []  # Contains internal nodes or leaf nodes

        exemplars = get_split(data_x, data_y, groups)

        closest_exemplars = get_closest_exemplars(exemplars, data_x, self.measure)
        for i, exemplar in enumerate(exemplars):
            # Find the data items that are closest to the exemplar (and thus having i in the closest_exemplars array)
            closest_data_x = data_x[closest_exemplars == i]
            closest_data_y = data_y[closest_exemplars == i]
            closest_data_groups = groups[closest_exemplars == i]

            subtree = get_node(closest_data_x, closest_data_y, closest_data_groups)
            self.branches.append(Branch(exemplar, i, subtree))  # TODO: Fix label somewhere here instead of i

    def predict(self, data):
        # Return the label of the closest subtree to the data point
        exemplar_distance = [self.measure(data, branch.exemplar) for branch in self.branches]
        closest_exemplars = get_closest_exemplars(exemplar_distance, [data], self.measure)
        branch = self.branches[closest_exemplars[0]]
        return branch.subtree.predict(data)

    def print(self, depth):
        for branch in self.branches:
            print(f"{'-' * depth}Exemplar ({branch.class_label}): " + str(branch.exemplar))
            branch.subtree.print(depth + 1)


# if all data reaching a node has the same class (node is pure), create_leaf function creates a new leaf node and assigns this class lbel to its field class
class LeafNode(TreeNode):
    def __init__(self, class_label):
        self.class_label = class_label  # This label is assigned to all data reaching this node

    def predict(self, data):
        return self.class_label

    def print(self, depth):
        print(f"{'-' * depth}Leaf ({self.class_label})")

In [189]:

# Splitting criteriaa
class ProximityTreeClassifier:
    def __init__(self):
        self.root = None

    def fit(self, data_x, data_y, groups):
        self.root = get_node(data_x, data_y, groups)
        self.num_features = data_x.shape[1]
        return self

    def predict(self, data):
        # Convert data to numpy array if not yet the case
        if not isinstance(data, np.ndarray):
            data = np.array(data)
        assert self.root is not None, "You must fit the model before predicting"
        assert data.shape[0] == self.num_features, "The number of features in the data must match the number of features in the training data"
        return self.root.predict(data)

    def print(self):
        self.root.print(0)

In [190]:
def is_pure(data_y):
    # Check if all data has the same class label
    unique_y = np.unique(data_y)
    return len(unique_y) == 1


def get_node(data_x, data_y, groups):
    if is_pure(data_y):
        class_label = data_y[0]
        return LeafNode(class_label)
    else:
        return InternalNode(data_x, data_y, groups)


# Some util functions
def random_pick_row(data):
    return data[np.random.randint(0, data.shape[0])]
    # return np.random.choice(group_x_with_label) # Can't use this for 2d arrays


def get_from_group_if_exists_else_random(group_x, group_y, data_x, data_y, label):
    # Get all elements of group_x where group_y == label
    group_x_with_label = group_x[group_y == label]
    if group_x_with_label.size != 0:
        # sample random with same label
        return random_pick_row(group_x_with_label)
    else:
        # If none exist, return a random element from data_x where data_y == label
        data_x_with_label = data_x[data_y == label]
        return random_pick_row(data_x_with_label)


def get_split(data_x, data_y, groups):
    # Sample uniformly any of the groups
    # Get the unique values in groups
    unique_groups = np.unique(groups)
    random_group = np.random.choice(unique_groups)

    # get the elements of data_x and data_y at the indices of the random group
    random_group_indices = np.where(groups == random_group)[0]
    random_group_x = data_x[random_group_indices]
    random_group_y = data_y[random_group_indices]

    exemplar_winner = get_from_group_if_exists_else_random(random_group_x, random_group_y, data_x, data_y, 1)
    exemplar_loser = get_from_group_if_exists_else_random(random_group_x, random_group_y, data_x, data_y, 0)
    exemplars = [exemplar_loser, exemplar_winner]

    return exemplars


def get_distance_to_exemplars(data_x_row, exemplars, measure):
    return [measure(data_x_row, exemplar) for exemplar in exemplars]


def get_closest_exemplars(exemplars, data, measure):
    # Return the index of the exemplar that is closest to the data point
    distances = [get_distance_to_exemplars(data_row, exemplars, measure) for data_row in data]
    closest_exemplar_indices = np.argmin(distances, axis=1)
    assert len(closest_exemplar_indices) == len(data)
    return closest_exemplar_indices


## Test
Just some random data example for debugging

In [191]:
import numpy as np
import matplotlib.pyplot as plt

data_x = np.array([[0, 1, 0], [1, 0, 1], [0, 0, 0],  # 3 headlines for test 0
                   [1, 0, 0], [1, 1, 0], [0, 1, 1]])  # 3 headlines for test 1

data_y = np.array([0, 1, 0,  # test 0
                   1, 0, 0])  # test 1

groups = np.array([0, 0, 0,  # test 0
                   1, 1, 1])  # test 1

In [192]:
model = ProximityTreeClassifier()
model.fit(data_x, data_y, groups)

model.print()

Exemplar (0): [1 1 0]
-Leaf (0)
Exemplar (1): [1 0 0]
-Exemplar (0): [0 0 0]
--Exemplar (0): [0 0 0]
---Leaf (0)
--Exemplar (1): [1 0 0]
---Leaf (1)
-Exemplar (1): [1 0 1]
--Leaf (1)


In [193]:
model.predict([1, 0, 0])

1

In [194]:
model.predict([0, 1, 1])

0

## More test
Let's take a larger test dataset from the real data

In [195]:
from util import get_wpm_train_test, get_manually_labeled_features

train_x, train_y, test_x, test_y, groups = get_wpm_train_test(include_groups=True, x_train_features_only=True,
                                                              full_y_test=True)
test_y_features = get_manually_labeled_features(test_x)

In [196]:
train_x_sm = train_x.head(7).to_numpy()
train_y_sm = train_y.head(7)['Winner'].astype(int).to_numpy()
groups_sm = groups.head(7).to_numpy()
train_x_sm

array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1],
       [1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1],
       [1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0]], dtype=int64)

In [197]:
model = ProximityTreeClassifier()
model.fit(train_x_sm, train_y_sm, groups_sm)
model.print()

Exemplar (0): [0 1 0 0 0 0 0 0 0 0 0 0 1 0 0]
-Exemplar (0): [1 1 0 0 1 0 0 0 0 0 0 0 0 1 0]
--Leaf (0)
-Exemplar (1): [1 1 0 0 1 0 0 0 0 0 0 1 0 0 1]
--Exemplar (0): [1 1 0 0 1 0 0 0 1 0 0 1 0 0 0]
---Leaf (0)
--Exemplar (1): [1 1 0 0 1 0 0 0 0 0 0 1 0 0 1]
---Leaf (1)
Exemplar (1): [1 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
-Exemplar (0): [1 0 0 0 1 0 0 0 0 0 0 0 1 0 1]
--Leaf (0)
-Exemplar (1): [1 0 0 0 0 0 1 0 0 0 0 0 1 0 0]
--Leaf (1)


In [198]:
# test_x_sm = test_x.head(4).to_numpy()
test_x_features_only = get_manually_labeled_features(test_x.head(4)).to_numpy()
test_y_sm = test_y.head(4)['Winner']
groups_test_sm = test_x.head(4)['Test'].to_numpy()

assert train_x_sm.shape[1] == test_x_features_only.shape[1]  # same number of features

test_x_features_only

array([[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
       [0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int64)

In [199]:
for x, y, group in zip(test_x_features_only, test_y_sm, groups_test_sm):
    prediction = model.predict(x)
    print(f'Test {group} prediction: {prediction}, actual: {y}')

Test 6 prediction: 0, actual: True
Test 6 prediction: 0, actual: False
Test 24 prediction: 1, actual: True
Test 24 prediction: 0, actual: False


This results in all the same predictions (all 0 or all 1) most of the time. (might be just coïncidence, it's also possible to predict the other number):

In [200]:
model.predict([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0])

0

## Forest

In [201]:
class ProximityForestClassifier:
    def __init__(self, n_trees=10):
        self.n_trees = n_trees
        self.trees = []

    def fit(self, data_x, data_y, groups):
        for i in range(self.n_trees):
            self.trees.append(ProximityTreeClassifier().fit(data_x, data_y, groups))
        return self

    def predict(self, data):
        tree_predictions = [self.trees[i].predict(data) for i in range(self.n_trees)]
        return np.bincount().argmax()

In [203]:
model = ProximityForestClassifier()
model.fit(train_x_sm, train_y_sm, groups_sm)
model.predict(test_x_features_only)

AssertionError: The number of features in the data must match the number of features in the training data

TODO: Continue from 3.3 Classifying with a Proximity Forest